# NYC Yellow Taxi - Bronze Layer Ingestion

## Overview

Initial Bronze layer ingestion for NYC Yellow Taxi trip data.

### Bronze Layer Principles

1. **Raw data preservation** - All columns cast to STRING
2. **Schema-on-read** - No type enforcement at ingestion
3. **Full lineage** - Source file tracking via `source_file` column
4. **No DQ filtering** - Data quality checks applied at Silver layer

---


## 1. Setup & Configuration


In [1]:
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, to_timestamp

from nyc_taxi_eta.configs.settings import GREEN_TAXI_DIR

# Create Spark session with Unity Catalog
# NOTE: Remove Delta extension - it conflicts with Unity Catalog's catalog routing
spark = (
    SparkSession.builder.appName("BronzeIngestion")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.jars.packages","io.delta:delta-spark_2.13:4.0.0,io.unitycatalog:unitycatalog-spark_2.13:0.3.0")
    # Unity Catalog configuration
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.delta.catalog.DeltaCatalog")
    .config("spark.sql.catalog.nyc_taxi", "io.unitycatalog.spark.UCSingleCatalog")
    .config("spark.sql.catalog.nyc_taxi.uri", "http://localhost:8080")
    .config("spark.sql.catalog.nyc_taxi.token", "")
    # Set nyc_taxi as default catalog
    .config("spark.sql.defaultCatalog", "nyc_taxi")
    # Performance configs
    .config("spark.driver.memory", "80g")
    .config("spark.sql.shuffle.partitions", "12")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.parquet.compression.codec", "snappy")
    .getOrCreate()
)

# Verify catalog is set correctly
print(f"Default catalog: {spark.catalog.currentCatalog()}")
print(f"Spark version: {spark.version}")

:: loading settings :: url = jar:file:/home/administrator/Desktop/datascience/github/nyc-taxi-eta/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/administrator/.ivy2.5.2/cache
The jars for the packages stored in: /home/administrator/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
io.unitycatalog#unitycatalog-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c48e672-4d31-4368-8db3-88098f7e80f7;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	found io.unitycatalog#unitycatalog-spark_2.13;0.3.0 in central
	found io.unitycatalog#unitycatalog-client;0.3.0 in central
	found org.slf4j#slf4j-api;2.0.13 in central
	found org.apache.logging.log4j#log4j-slf4j2-impl;2.24.3 in central
	found org.apache.logging.log4j#log4j-api

Default catalog: nyc_taxi
Spark version: 4.0.0


## 2. Schema Definition


In [7]:
paths = [str(p) for p in GREEN_TAXI_DIR.glob("green_tripdata_2014*.parquet")]
def clean_column_names(col_name):
    return col_name.strip().lower().replace(" ", "_").replace("-", "_")
df = spark.read.parquet(*paths)
df = df.toDF(*[clean_column_names(c) for c in df.columns])
df = df.withColumn(
    "year", year(to_timestamp(col("lpep_pickup_datetime")))
)
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+----+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|year|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+----+
|       2| 2014-02-01 00:05:44|  2014-02-01 00:05:50|                 N|         1|         146|         146|              1|          0.0|        2

In [8]:
from nyc_taxi_eta.configs.settings import BRONZE_DIR
try:
    start_time = datetime.now()

    catalog = "nyc_taxi"
    schema = "bronze"
    table = "green_taxi_trips"
    full_table_name = f"{catalog}.{schema}.{table}"

    # Verify catalog
    print(f"Current catalog: {spark.catalog.currentCatalog()}")

    (
        df.write
            .format("delta")
            .mode("overwrite")
            .partitionBy("year")
            .option("path", f"{BRONZE_DIR}/{table}")
            .saveAsTable(full_table_name)
    )


    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    print(f"✅ Table written: {full_table_name}")
    print(f"⏱️  Duration: {duration:.1f} seconds")


except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

Current catalog: nyc_taxi


25/11/30 19:56:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


✅ Table written: nyc_taxi.bronze.green_taxi_trips
⏱️  Duration: 18.4 seconds


In [9]:
# Stop Spark
spark.stop()
print("Done!")

Done!
